In [1]:
from operator import itemgetter
# from llama_cpp import Llama

from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import LlamaCppEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.vectorstores import FAISS
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain.llms import CTransformers
# from langchain.embeddings import LlamaCppEmbeddings

In [2]:
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llama_model_path = "../../models/zephyr-7b-beta.Q4_K_M.gguf"

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=llama_model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ../../models/zephyr-7b-beta.Q4_K_M.gguf (version unknown)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     

In [3]:
# config = {'max_new_tokens': 1024, 'repetition_penalty': 1.1}

# llm = CTransformers(
#     model="TheBloke/zephyr-7B-alpha-GGUF",
#     model_file="zephyr-7b-alpha.Q4_K_M.gguf",
#     model_type="mistral",
#     gpu_layers=50,
#     config=config,
# )
#Use Llama model for embedding
llama_model_path = "../../models/zephyr-7b-beta.Q4_K_M.gguf"
embeddings = LlamaCppEmbeddings(model_path=llama_model_path, n_ctx=2048)
# llm = Llama(model_path=llama_model_path)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ../../models/zephyr-7b-beta.Q4_K_M.gguf (version unknown)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     

In [4]:
output = llm("Q: Name the planets in the solar system? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)

1) Mercury, 2) Venus, 3) Earth, 4) Mars, 5) Jupiter, 6) Saturn


llama_print_timings:        load time =  5175.20 ms
llama_print_timings:      sample time =    32.61 ms /    32 runs   (    1.02 ms per token,   981.26 tokens per second)
llama_print_timings: prompt eval time =  5175.10 ms /    14 tokens (  369.65 ms per token,     2.71 tokens per second)
llama_print_timings:        eval time =  1085.39 ms /    31 runs   (   35.01 ms per token,    28.56 tokens per second)
llama_print_timings:       total time =  6363.49 ms


In [5]:
output

'1) Mercury, 2) Venus, 3) Earth, 4) Mars, 5) Jupiter, 6) Saturn'

In [6]:
vectorstore = FAISS.from_texts(["harrison worked at kensho"], embedding=embeddings)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = llm


llama_print_timings:        load time =   522.65 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =   522.04 ms /     8 tokens (   65.26 ms per token,    15.32 tokens per second)
llama_print_timings:        eval time =    70.40 ms /     1 runs   (   70.40 ms per token,    14.20 tokens per second)
llama_print_timings:       total time =   594.88 ms


In [7]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [8]:
chain.invoke("where did harrison work?")



llama_print_timings:        load time =   522.65 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =   391.70 ms /     8 tokens (   48.96 ms per token,    20.42 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   393.17 ms
Llama.generate: prefix-match hit



Answer: Harrison worked at Kensho.

Context: This statement is a fact, and we can infer that Harrison's job was at the company Kensho. The context provided is sufficient to answer the question accurately.


llama_print_timings:        load time =  5175.20 ms
llama_print_timings:      sample time =    44.27 ms /    50 runs   (    0.89 ms per token,  1129.33 tokens per second)
llama_print_timings: prompt eval time =   428.84 ms /    42 tokens (   10.21 ms per token,    97.94 tokens per second)
llama_print_timings:        eval time =  1748.57 ms /    49 runs   (   35.69 ms per token,    28.02 tokens per second)
llama_print_timings:       total time =  2319.99 ms


"\nAnswer: Harrison worked at Kensho.\n\nContext: This statement is a fact, and we can infer that Harrison's job was at the company Kensho. The context provided is sufficient to answer the question accurately."

In [9]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever, 
    "question": itemgetter("question"), 
    "language": itemgetter("language")
} | prompt | model | StrOutputParser()

In [10]:
chain.invoke({"question": "where did harrison work", "language": "italian"})


llama_print_timings:        load time =   522.65 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =   387.42 ms /     7 tokens (   55.35 ms per token,    18.07 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   388.37 ms
Llama.generate: prefix-match hit



Possible Answers: Dove ha lavorato Harrison?

In the following scenario:
The question is posed by an Italian speaker to a translator, who must provide an accurate and clear answer in Italian based on the provided context. The translator must not assume any prior knowledge or context beyond what is explicitly stated in the document. The translator's response should accurately convey the meaning of "where did harrison work" in the given context.


llama_print_timings:        load time =  5175.20 ms
llama_print_timings:      sample time =    90.77 ms /    99 runs   (    0.92 ms per token,  1090.69 tokens per second)
llama_print_timings: prompt eval time =   249.36 ms /    12 tokens (   20.78 ms per token,    48.12 tokens per second)
llama_print_timings:        eval time =  3489.24 ms /    98 runs   (   35.60 ms per token,    28.09 tokens per second)
llama_print_timings:       total time =  4033.68 ms


'\nPossible Answers: Dove ha lavorato Harrison?\n\nIn the following scenario:\nThe question is posed by an Italian speaker to a translator, who must provide an accurate and clear answer in Italian based on the provided context. The translator must not assume any prior knowledge or context beyond what is explicitly stated in the document. The translator\'s response should accurately convey the meaning of "where did harrison work" in the given context.'

In [11]:
llm("Simulate a rap battle between Stephen Colbert and John Oliver")

Llama.generate: prefix-match hit


 on current events. Include specific references to recent news stories, use vivid language and poetic devices such as metaphors and rhymes, and aim for a humorous and entertaining tone. Bonus points for incorporating insights from political science or social commentary into the lyrics.


llama_print_timings:        load time =  5175.20 ms
llama_print_timings:      sample time =    56.46 ms /    59 runs   (    0.96 ms per token,  1045.02 tokens per second)
llama_print_timings: prompt eval time =   263.43 ms /    12 tokens (   21.95 ms per token,    45.55 tokens per second)
llama_print_timings:        eval time =  2023.52 ms /    58 runs   (   34.89 ms per token,    28.66 tokens per second)
llama_print_timings:       total time =  2467.48 ms


' on current events. Include specific references to recent news stories, use vivid language and poetic devices such as metaphors and rhymes, and aim for a humorous and entertaining tone. Bonus points for incorporating insights from political science or social commentary into the lyrics.'